# S1: Insertion-Only Dynamic Connectivity in General Disk Graphs\[1\]

*Author: Lars Nitzschke* 

This notebook is part of the work in the seminar **Seminar on Algorithms**. More precisely it implements the algorithms and data structure discussed in the paper **"Insertion-Only Dynamic Connectivity in General Disk Graphs"** by Kaplan et al.[1]
The notebook offers interactive visualisations and animations.

## Table of Contents

1. Introduction
1. Setup
1. Implementation
1. References  

## 1. Introduction

This notebook implements and presents the approach by Kaplan et al. [1] They give a solution to the following problem: Given a set $S \subseteq \mathbb{R}^2$ of $n$ sites in a plane, where every site $s\in S$ has an associated radius $r_s > 0$, be $\mathcal{D}(S)$ the disk intersection graph. Its vertices are the sites $s\in S$ and there exists an edge between two sites $s,t$, iff the two disks at $s,t$ with radii $r_s, r_t$ intersect. A connectivity query then asks whether two given disks are in the same connected component.

The presented approach requires $O(\log^6 n)$ expected amortized time for adding a new disk and $O(\alpha(n))$ amortized time querying the connected component for a given disk. For that they introduce a new data structure, the component tree $\mathcal{T}_C$ and the insertion algorithm for adding a new disk to the tree. Both are presented in section 3 of this notebook.

The detailed describtion of the algorithms can be found in the paper itself and the report on it written for the seminar.

## 2. Setup

First let's do some setup. This is not very interesting, so you can skip to Section 3 if you want.

We now import everything we'll need throughout this notebook from external sources, including our module for generic data structures, our module for geometric primitives and operations as well as our module for visualisation purposes. 
These modules are explained in [notebook no. 00](./00-Basics.ipynb).

In [1]:
# Python standard library imports
from abc import ABC, abstractmethod
from typing import Any, Optional, Iterable
import random

# Data structure, geometry and visualisation module imports
from modules.data_structures import Comparator, ComparisonResult as CR, HalfEdge, DoublyConnectedEdgeList, PointLocation, Face, VDLineSegment, AnimationBinaryTreeDict, PLSearchStructure
from modules.geometry import Point, PointReference, PointSequence, LineSegment, EPSILON, Rectangle, VerticalOrientation as VORT, HorizontalOrientation as HORT
from modules.visualisation import VisualisationTool, PointLocationInstance, VerticalExtensionMode, PointLocationMode, DiskMode, DiskSetInstance, LineSegmentsMode, DiskConnectivityInstance
from modules.geometry import Disk
from modules.data_structures import DisjointSet, ComponentTree, DiskConnectivity
from pyquadtree import QuadTree

Component = list[Disk]

Additionally, we create an object for our visualisation tool. <!-- and register a few example instances.-->

In [2]:
visualisation_tool = VisualisationTool(400, 400, DiskConnectivityInstance())
canvas_size = min(visualisation_tool.width, visualisation_tool.height)

## 3. Implementation

In this section the approach of Kaplan et al. [1] is implemented.

### 3.1 Data Structure and insertion algorithm

We start with the data structure, the component tree $\mathcal{T}_C$. The full implementation can be found in the module `data_structures.component_tree`. Here we look at the interesting parts of the implementation. The "_" at the end of the class names is for differentiating from the acutal implementation.

The component tree consisits of two parts. Firstly, the root node. The tree is build from those nodes which are implemented on the bottom. Secondly, the leaf queue, holding all empty leafs.

In [3]:
class ComponentTreeNode_:
    def __init__(self):
        self._component: Optional[list[Disk]] = None  # None for every node but the leaves
        self._awnn = QuadTree(bbox=(0, 0, 1000, 1000))
        self._left: Optional[ComponentTreeNode_] = None
        self._right: Optional[ComponentTreeNode_] = None
        self._parent: Optional[ComponentTreeNode_] = None
        self._level: int = 0
    
    def query_intersected(self, disk: Disk):
        if self.is_leaf():
            return [self]
        intersected: list[ComponentTreeNode_] = []
        # Left
        left_nearest = self._left._awnn.nearest_neighbors(disk.center_point.as_tuple())
        left_nearest = left_nearest[0] if left_nearest != [] else None
        if left_nearest is not None and disk.center_point.distance(left_nearest.item.center_point) <= disk.radius + left_nearest.item.radius:
            # If result not empty recurse into the child.
            intersected.extend(self._left.query_intersected(disk))
        # Right
        right_nearest = self._right._awnn.nearest_neighbors(disk.center_point.as_tuple())
        right_nearest = right_nearest[0] if right_nearest != [] else None
        if right_nearest is not None and disk.center_point.distance(right_nearest.item.center_point) <= disk.radius + right_nearest.item.radius:
            intersected.extend(self._right.query_intersected(disk))

        return intersected

The AWNNs are realised with a quadtree, which is capable of efficienty nearest neighbor queries. However, it cannot do weighted nearest neighbors as necessary for general disks. Thats why this implementation only works on unit disks. To complete it to solve the actual problem, one would only need to exchange this data structure.

The interesting part for the nodes is the querying algorithm. As described in the paper, the query works like this:
- If a leaf is reached, that leaf is returned. The reason why the leaf itself is returned and not the stored component is that during insertion the algorithm needs to traverse the tree starting from the leaf.
- Otherwise, both the left and right subtree's AWNNs are queried for the closest neighbor. If either intersects the querying disk, the procedure recurses into the respective child. Results are collected in a list and returned.

In [4]:
class ComponentTree_:
    def __init__(self):
        self._root = ComponentTreeNode_()
        self._leafQueue = [self._root]

    def insert_component(self, component: list[Disk]):
        # 1 Get Leaf
        if self._leafQueue == []:
            # Double the size of the tree
            new_node = ComponentTreeNode_()
            new_node.set_left_child(self._root)
            new_node.set_right_child(self._full_sub_tree(depth = self._root._level + 1))
            new_node._level = self._root._level + 1
            for disk_element in self._root._awnn.get_all_elements():
                new_node._awnn.add(disk_element.item, disk_element.point)
            self._root = new_node
        leaf: ComponentTreeNode_ = self._leafQueue.pop(0)

        # 2 Insert component into the leaf
        leaf._component = component            
        for disk in component:
            leaf._awnn.add(disk, disk.center_point.as_tuple())

        # Update awnns on the path to the root
        current_node: ComponentTreeNode_ = leaf
        while current_node._parent is not None:
            current_node = current_node._parent
            for disk in component:
                current_node._awnn.add(disk, disk.center_point.as_tuple())

    def query(self, disk: Disk) -> list[Component]:
        return [node._component for node in self.query_nodes(disk)]

    def query_nodes(self, disk: Disk):
        # Query root
        nearest = self._root._awnn.nearest_neighbors(disk.center_point.as_tuple())
        if len(nearest) == 0:
            return []  # No intersected components
        nearest = nearest[0]
        if disk.center_point.distance(nearest.item.center_point) > disk.radius + nearest.item.radius:
            return []  # No intersected components
        return self._root.query_intersected(disk)

    
    def insert_disk(self, disk: Disk):
        intersected_component_leafs = self.query_nodes(disk)
        if intersected_component_leafs == []:
            # Insert singleton component
            self.insert_component([disk])
        else:
            # find largest component and insert component
            largest_component_leaf: ComponentTreeNode_ = max(intersected_component_leafs, key=lambda node: len(node._component))
            largest_component_leaf._component.append(disk)
            largest_component_leaf._awnn.add(disk, disk.center_point.as_tuple())

            # Update awnns on the path to the root
            current_node: ComponentTreeNode_ = largest_component_leaf
            while current_node._parent is not None:
                current_node = current_node._parent
                current_node._awnn.add(disk, disk.center_point.as_tuple())

            # if more than one intersected component: clean-up step
            if len(intersected_component_leafs) > 1:
                # for every component but the largest one, find lca and update awnns on the path up and down 
                for component_node in intersected_component_leafs:
                    if component_node is largest_component_leaf:
                        continue
                    for site in component_node._component:
                        largest_component_leaf._component.append(site)
                    # Start with both nodes (largest component and the other intersected one)
                    # they are always at the same level, as we start at leafs of a full tree.
                    node_l, node_o = largest_component_leaf, component_node
                    while node_l is not node_o: # while not at the same node (lca), update both: insertions and deletions 
                        for component_disk in component_node._component:
                            node_l._awnn.add(component_disk, component_disk.center_point.as_tuple())
                            node_o._awnn.delete(component_disk)
                        node_l = node_l._parent
                        node_o = node_o._parent

                    # empty the other node
                    component_node._component = None
                    self._leafQueue.append(component_node)

From the nodes the tree is built in the class `ComponentTree_`. In here is also where the algorithm for inserting a new disk is implemented.

As in the paper the simple case is inserting a new component, which is implemented in lines 6-29. It starts by checking whether there is an available leaf and if necessary increases the tree size (lines 6-17). The component is simply inserted into the first available leaf and also all sites are inserted into the AWNN of the leaf (lines 20-22).
Finally the AWNNs on the path to the root are updated by traversing the tree using the `_parent` node references.

Querying just returns all components of the leafs returned from the already discussed procedure.

It remains to look at the implementation of inserting a disk. It starts by checking if it is a singleton component and if so inserts it that way (lines 47-49). Otherwise the new disk is inserted into the largest one of the intersected components (lines 51-54). The AWNNs are updated acordingly, similar to inserting a full component (lines 57-60). Afterwards the clean-up step is performed. The sites are moved to the largest one from the other component (lines 68-69 and 81-82). Then the same is done on the path to their lowest common anchestor (lines 72-78). The loop has a reference to both nodes. As the tree is complete, the nodes are always on the same level and traversing upwards can be done simultaneously.

### 3.2 Full insertion algorithm and dynamic connectivity

It remains to put the insertion component tree together with the disjoint set data structure. We start with a simple (inefficient) connectivity, that checks all intersections and stores them in the data structure `data_structures.disjoint_set`.

In [5]:
def disjoint_set_connectivity(disk_instance: DiskConnectivity) -> PointSequence:
    disk_set = disk_instance._disk_set
    point_seq = PointSequence()  # Only for visualisation
    disjoint_set = DisjointSet[Disk]()
    for disk in disk_set:
        disjoint_set.make_set(disk)
    for disk in disk_set:
        point_seq.append(PointReference([disk.center_point, Point(disk.center_point.x + disk.radius, disk.center_point.y)], 0, disk.label))
        for other_disk in disk_set:
            point_seq.animate(PointReference([other_disk.center_point, Point(other_disk.center_point.x + other_disk.radius, other_disk.center_point.y)], 0, other_disk.label))
            if disk == other_disk:
                continue
            if disk.intersects(other_disk):
                disjoint_set.union(disk, other_disk)
        point_seq.pop()
    return point_seq

visualisation_tool.register_algorithm("Disjoint Set Connectivity", disjoint_set_connectivity, LineSegmentsMode(vertex_radius = 0, highlight_radius = 0))

For every disk, a set is created and then all pairs of disks are checked for intersection. If they do intersect, they are unioned into the same set. The set could now be used to query whether two disks are in the same component, by querying the representative each. If it is the same, they are in the same component.

In [ ]:
def component_tree_connectivity(disk_instance: DiskConnectivity) -> PointSequence:
    point_seq = PointSequence()
    disk_instance._component_tree = ComponentTree()  #Actual implementation of the component tree
    disjoint_set = DisjointSet[Disk]()
    disk_list = list(disk_instance._disk_set)
    if len(disk_list) == 0:
        return point_seq
    disk_instance._excluded_disk = disk_list[0] 
    disks_to_add = disk_list[1:]
    for disk in disks_to_add:
        disjoint_set.make_set(disk)
        connected_components = disk_instance._component_tree.query(disk)
        for connected_component in connected_components:  # merge all connected components
            disjoint_set.union(disk, connected_component[0])   
        disk_instance._component_tree.insert_disk(disk)

    visualisation_tool._tree_ui_1.value = disk_instance._component_tree.html()
    return point_seq

def component_tree_add_disk(disk_instance: DiskConnectivity) -> PointSequence:
    point_seq = PointSequence()
    if len(disk_instance._disk_set) == 0:
        return point_seq
    point_seq.append(PointReference([disk_instance._excluded_disk.center_point, Point(disk_instance._excluded_disk.center_point.x + disk_instance._excluded_disk.radius, disk_instance._excluded_disk.center_point.y)], 0, disk_instance._excluded_disk.label))
    disk_instance._disjoint_set.make_set(disk_instance._excluded_disk)
    connected_components = disk_instance._component_tree.query(disk_instance._excluded_disk)
    for connected_component in connected_components:  # merge all connected components
        disk_instance._disjoint_set.union(disk_instance._excluded_disk, connected_component[0])
        point_seq.animate(PointReference([connected_component[0].center_point, connected_component[0].center_point], 0, connected_component[0].label))
    disk_instance._component_tree.insert_disk(disk_instance._excluded_disk)
    disk_instance._excluded_disk = None
    visualisation_tool._tree_ui_2.value = disk_instance._component_tree.html()
    return point_seq
    


visualisation_tool.register_algorithm("Component Tree Connectivity", component_tree_connectivity, LineSegmentsMode(vertex_radius = 0, highlight_radius = 0))
visualisation_tool.register_algorithm("Component Tree Add Disk", component_tree_add_disk, DiskMode(), preprocessing=component_tree_connectivity)
visualisation_tool.display()

Now for the implementation using the tree. The first method `component_tree_connectivity` builds the disjoint set for querying and also the component tree. Thereby it excludes one disk from the set which is then used by the second method `component_tree_add_disk` for inserting that into the tree separately. Which disk that is, is random, as the disks are stored in a set which is unordered.

When inserting a disk, the tree is queried for intersected components. For those components, union-operations are performed on the disjoint set. Afterwards, the disk is inserted into the tree.
The states of the component trees before and after the insertion are visualised in a simple text form.

## 4. References

\[1\] Haim Kaplan and Katharina Klost and Kristin Knorr and Wolfgang Mulzer and Liam Roditty. **Insertion-Only Dynamic Connectivity in General Disk Graphs.**, in 2024 Symposium on Simplicity in Algorithms (SOSA), doi: 10.1137/1.9781611977936.27